<a href="https://colab.research.google.com/github/nelsonfrugeri-tech/playground-generative-ai/blob/master/logits_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import login
login(new_session=False)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
print(torch.cuda.is_available())  # Deve imprimir: True
print(torch.cuda.get_device_name(0))  # Deve mostrar: 'NVIDIA A100-SXM4-40GB'


In [ ]:
MODEL_ID = "google/gemma-3-4b-pt"

In [ ]:
from transformers import LogitsProcessor, pipeline
from transformers.utils import add_start_docstrings
from transformers.generation.logits_process import LOGITS_PROCESSOR_INPUTS_DOCSTRING
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np

In [ ]:
class RulesLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer, palavras_proibidas):
        self.tokenizer = tokenizer
        # Converte cada palavra proibida em lista de token_ids
        self.tokens_proibidos = set()
        for palavra in palavras_proibidas:
            for tid in tokenizer.encode(palavra, add_special_tokens=False):
                self.tokens_proibidos.add(tid)

    def apply_rules(self, seq):
        return not any(palavra in seq for palavra in palavras_proibidas)

    def __call__(self, input_ids: torch.LongTensor, input_logits: torch.FloatTensor) -> torch.FloatTensor:
        output_logits = input_logits.clone()
        eos_token_id = tokenizer.eos_token_id

        for idx, input_id in enumerate(input_ids):
            seq = self.tokenizer.decode(input_id)
            if not self.apply_rules(seq):
                # Permite sempre ao menos o EOS
                output_logits[idx] = -float("inf")
                output_logits[idx, eos_token_id] = 0

        # Também bloqueia qualquer próximo token proibido:
        output_logits[:, list(self.tokens_proibidos)] = -float("inf")
        return output_logits

In [ ]:
# Carrega modelo e tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID)

In [ ]:
# Defina uma lista de palavras proibidas para a regra (exemplo simples)
palavras_proibidas = ["desculpe", "infelizmente"]

# Cria pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

rules_processor = MyRulesLogitsProcessor(tokenizer, palavras_proibidas)

# Defina o prompt de entrada
input_message = (
    "<|system|>\n"
    "Você é um atendente cordial, direto e focado na solução. "
    "Explique ao cliente o que será feito para resolver o problema. Seja transparente e objetivo. \n\n"
    "Abaixo segue informações do sistema de logística. \n\n"
    "Consulta ao sistema logístico: status do pedido #12345 — Pedido ATRASADO. Motivo: alto volume de entregas na região. Previsão de nova entrega: 2 dias úteis.\n\n"
    "<|user|>\n"
    "Meu pedido está atrasado e eu preciso de uma solução urgente. Não aceito mais desculpas, quero uma resposta clara e objetiva sobre o que será feito.\n"
    "<|assistant|>\n"
)


# Executa a geração
results = pipe(
    input_message,
    max_new_tokens=1024,
    do_sample=True,
    temperature=1.0,
    num_beams=5,
    logits_processor=[rules_processor],
    pad_token_id=tokenizer.eos_token_id
)

print("Resposta gerada:\n", results)

In [ ]:
from typing import Dict
from abc import ABC, abstractmethod
from transformers import LogitsProcessor, PreTrainedTokenizerBase
import torch


class MaskingRules(ABC):
    @abstractmethod
    def get_tokenid2prob(self) -> Dict[int, float]:
      pass


class MaskingRulesImpl(MaskingRules):
    def __init__(self, tokenizer: PreTrainedTokenizerBase, regras_dict: Dict[str, float]):
        self.tokenid2prob: Dict[int, float] = {}
        for termo, prob in regras_dict.items():
            token_ids = tokenizer.encode(termo, add_special_tokens=False)
            for tid in token_ids:
                self.tokenid2prob[tid] = -float("inf") if prob == 0 else float(prob)

    def get_tokenid2prob(self) -> Dict[int, float]:
        return self.tokenid2prob


class RulesLogitsProcessor(LogitsProcessor):
    def __init__(self, masking_rules: MaskingRules):
        self.tokenid2prob: Dict[int, float] = masking_rules.get_tokenid2prob()

    def __call__(
        self,
        input_ids: torch.LongTensor,
        scores: torch.FloatTensor
    ) -> torch.FloatTensor:
        output_logits: torch.FloatTensor = scores.clone()
        for tid, prob in self.tokenid2prob.items():
            output_logits[:, tid] += prob
        return output_logits


In [ ]:
from typing import Dict
from transformers import pipeline, PreTrainedTokenizerBase, AutoModelForCausalLM, AutoTokenizer

# Defina regras com dicionário: termo -> probabilidade
rules: Dict[str, float] = {
    "New Balance": 3,    # Incentiva fortemente a marca New Balance
    "Asics": 2,          # Incentiva fortemente a marca Asics
    "Adidas": -3,        # Desincentiva Adidas, menor relevância
    "Nike": -3           # Desincentiva Nike, menor relevância
}

# Instancia MaskingRules e o processor
rules_processor = RulesLogitsProcessor(MaskingRulesImpl(tokenizer, rules))

# Cria pipeline Hugging Face
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

# Prompt realista de compliance regulatório financeiro
input_message = (
    """
    <bos><start_of_turn>system
    Você é um assistente virtual especializado em recomendar tênis esportivos para clientes de e-commerce.
    Recomende os tênis do nosso catálogo:
    Nike
    Adidas
    Asics
    New Balance
    Puma
    Under Armour
    Reebok
    Mizuno
    Saucony
    Brooks
    ---
    De acordo com seus conhecimentos recomende o melhor tênis dentro do nosso catálogo para o cliente.
    Escreva na lingua portuguesa do Brasil.
    Gere uma resposta em linguagem natural, lembre-se que você está conversando com um humano.
    <end_of_turn>
    <start_of_turn>user
    Quero comprar um bom tênis de corrida para longas distâncias, pode me recomendar algumas opções?
    <end_of_turn>
    <start_of_turn>model
    """
)

# Executa a geração
results = pipe(
    input_message,
    max_new_tokens=110,
    do_sample=True,
    temperature=0.7,
    num_beams=5,
    logits_processor=[rules_processor],
    pad_token_id=tokenizer.eos_token_id
)

print("Resposta gerada:\n", results[0]["generated_text"])
